In [545]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import unicodedata
from features import *

import string
import re
import spacy
from datasets import load_from_disk
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from utils import preprocess
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report



In [546]:
word2vec_model = api.load('word2vec-google-news-300')

In [547]:
wikiqa_data = load_from_disk("wikiqa")
test_data_set = wikiqa_data["test"]
train_data_set = wikiqa_data["train"]
validation_data_set = wikiqa_data["validation"]


def preprocess_examples(examples):
    examples['question'] = [preprocess(q) for q in examples['question']]
    examples['answer'] = [preprocess(a) for a in examples['answer']]
    return examples

train_data_set = train_data_set.map(preprocess_examples, batched=True)
validation_data_set = validation_data_set.map(preprocess_examples, batched=True)
test_data_set = test_data_set.map(preprocess_examples, batched=True)

# Convertir en DataFrame
train_df = pd.DataFrame({
    'question': train_data_set['question'],
    'answer': train_data_set['answer'],
    'label': train_data_set['label']
})

validation_df = pd.DataFrame({
    'question': validation_data_set['question'],
    'answer': validation_data_set['answer'],
    'label': validation_data_set['label']
})

test_df = pd.DataFrame({
    'question': test_data_set['question'],
    'answer': test_data_set['answer'],
    'label': test_data_set['label']
})

Map: 100%|██████████| 20360/20360 [00:00<00:00, 55554.24 examples/s]


Map: 100%|██████████| 6165/6165 [00:00<00:00, 56464.06 examples/s]


In [548]:
# Supprimer les questions sans réponse pertinente
def filter_non_relevant(df):
    relevant_questions = df[df['label'] == 1]['question'].unique()
    return df[df['question'].isin(relevant_questions)]

train_df = filter_non_relevant(train_df)
validation_df = filter_non_relevant(validation_df)
test_df = filter_non_relevant(test_df)

In [549]:
# Regrouper les réponses et les labels pour chaque question
def group_answers(df):
    grouped = df.groupby('question').agg(list).reset_index()
    return grouped

train_grouped = group_answers(train_df)
validation_grouped = group_answers(validation_df)
test_grouped = group_answers(test_df)

### embeddings

In [550]:
import gensim.downloader as api

model = api.load("word2vec-google-news-300")

def embed_sentence(sentence):
    words = sentence.split()
    word_vecs = [model[word] for word in words if word in model]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

train_grouped['question_vec'] = train_grouped['question'].apply(embed_sentence)
train_grouped['answer_vecs'] = train_grouped['answer'].apply(lambda answers: [embed_sentence(a) for a in answers])

validation_grouped['question_vec'] = validation_grouped['question'].apply(embed_sentence)
validation_grouped['answer_vecs'] = validation_grouped['answer'].apply(lambda answers: [embed_sentence(a) for a in answers])

test_grouped['question_vec'] = test_grouped['question'].apply(embed_sentence)
test_grouped['answer_vecs'] = test_grouped['answer'].apply(lambda answers: [embed_sentence(a) for a in answers])

In [551]:
def filter_dataframe(df):
    # Filtrer les lignes avec au moins 4 labels ou plus
    filtered_df = df[df['label'].apply(lambda x: len(x) >= 4)]
    return filtered_df

train_grouped_filtered = filter_dataframe(train_grouped)

validation_grouped_filtered = filter_and_limit_responses(validation_grouped)

test_grouped_filtered = filter_and_limit_responses(test_grouped)

In [552]:
import pandas as pd

import pandas as pd

def filter_and_limit_responses(df):
    # Fonction pour filtrer et limiter les réponses par question
    new_rows = []

    for _, row in df.iterrows():
        question = row['question']
        answers = row['answer']
        labels = row['label']
        question_vec = row['question_vec']
        answer_vecs = row['answer_vecs']

        # Filtrer les réponses ayant des labels 1
        filtered_answers = [ans for ans, lbl in zip(answers, labels) if lbl == 1]
        filtered_labels = [lbl for lbl in labels if lbl == 1]
        filtered_answer_vecs = [vec for vec, lbl in zip(answer_vecs, labels) if lbl == 1]

        # Si moins de 4 réponses, ajouter des réponses ayant des labels 0
        if len(filtered_answers) < 4:
            for ans, lbl, vec in zip(answers, labels, answer_vecs):
                if lbl == 0 and len(filtered_answers) < 4:
                    filtered_answers.append(ans)
                    filtered_labels.append(lbl)
                    filtered_answer_vecs.append(vec)

        # Si plus de 4 réponses, couper les réponses supplémentaires
        filtered_answers = filtered_answers[:4]
        filtered_labels = filtered_labels[:4]
        filtered_answer_vecs = filtered_answer_vecs[:4]

        # Ajouter la nouvelle ligne au dataframe
        new_rows.append({
            'question': question,
            'answer': filtered_answers,
            'label': filtered_labels,
            'question_vec': question_vec,
            'answer_vecs': filtered_answer_vecs
        })

    # Créer le nouveau dataframe
    new_df = pd.DataFrame(new_rows)
    return new_df



# Exemple d'utilisation
# Remplacez `your_dataframe` par le nom de votre DataFrame
test = filter_and_limit_responses(train_grouped_filtered)
print(test)

train_grouped_clean = filter_and_limit_responses(train_grouped_filtered)

validation_grouped_clean = filter_and_limit_responses(validation_grouped_filtered)

test_grouped_clean = filter_and_limit_responses(test_grouped_filtered)

                                              question  \
0                            how a rocket engine works   
1                how are aircraft radial engines built   
2    how are cholera and typhus transmitted and pre...   
3                         how are glacier caves formed   
4    how are the of electrons in each shell determined   
..                                                 ...   
751  who wrote the song a little more country than ...   
752                         who wrote the song cocaine   
753                  who wrote the song feelin alright   
754                    who wrote whats my name rihanna   
755                          who wrote white christmas   

                                                answer         label  \
0    [a rocket engine or simply rocket is a jet eng...  [1, 0, 0, 0]   
1    [the radial engine is a reciprocating type int...  [1, 0, 0, 0]   
2    [transmission occurs primarily by drinking wat...  [1, 0, 0, 0]   
3    [a glacier

In [553]:
train_grouped_clean

,question,answer,label,question_vec,answer_vecs
0,how a rocket engine works,[a rocket engine or simply rocket is a jet eng...,"[1, 0, 0, 0]","[0.15490723, 0.11816406, -0.011108398, -0.0439...","[[0.08710734, 0.059940156, 0.022359213, 0.0036..."
1,how are aircraft radial engines built,[the radial engine is a reciprocating type int...,"[1, 0, 0, 0]","[0.06514486, 0.12573242, 0.091430664, 0.088806...","[[0.052048393, 0.07698959, 0.046286542, 0.0554..."
2,how are cholera and typhus transmitted and pre...,[transmission occurs primarily by drinking wat...,"[1, 0, 0, 0]","[0.052001953, 0.056559246, 0.06315104, 0.05025...","[[-0.007805718, 0.05677626, 0.059021562, 0.109..."
3,how are glacier caves formed,[a glacier cave is a cave formed within the ic...,"[1, 0, 0, 0]","[-0.008984375, 0.07998047, 0.045800783, 0.0598...","[[-0.09524536, 0.008273655, 0.020100912, 0.053..."
4,how are the of electrons in each shell determined,[each shell can contain only a fixed number of...,"[1, 0, 0, 0]","[0.08529663, 0.0027503967, 0.048797607, -0.006...","[[0.09835568, -0.030860128, 0.06585508, 0.0438..."
...,...,...,...,...,...
751,who wrote the song a little more country than ...,[a little more country than that is the title ...,"[1, 0, 0, 0]","[0.063463, -0.010219998, 0.01894294, 0.0960286...","[[-0.03970602, -0.0020380435, 0.015831325, 0.1..."
752,who wrote the song cocaine,[cocaine is a song written and recorded by jj ...,"[1, 0, 0, 0]","[0.12753907, -0.020837402, -0.005395508, 0.025...","[[0.0038231744, 0.021272447, 0.064197116, 0.05..."
753,who wrote the song feelin alright,[feelin alright also known as feeling alright ...,"[1, 0, 0, 0]","[0.09956869, -0.004018148, 0.036051434, 0.0548...","[[0.05891087, -0.0075586983, 0.005429475, 0.07..."
754,who wrote whats my name rihanna,[the rb song was produced by the norwegian pro...,"[1, 0, 0, 0]","[0.04353841, 0.010345459, 0.03680293, 0.114705...","[[0.02389249, 0.042089287, 0.034859397, 0.0190..."


### test de CNN

### BY RANKING 

In [554]:
def prepare_data(df):
    q_data = []
    a_data = []
    y_data = []

    for _, row in df.iterrows():
        question_vec = np.array(row['question_vec'])
        answer_vecs = np.array(row['answer_vecs'])
        labels = row['label']

        num_answers = len(answer_vecs)
        q_data.extend([question_vec] * num_answers)
        a_data.extend(answer_vecs)
        y_data.extend(labels)

    q_data = np.expand_dims(np.array(q_data), axis=1)  # Ajouter une dimension pour l'axe de séquence
    a_data = np.expand_dims(np.array(a_data), axis=1)  # Ajouter une dimension pour l'axe de séquence
    y_data = np.array(y_data)

    return q_data, a_data, y_data

q_train, a_train, y_train = prepare_data(train_grouped_clean)

# Afficher les formes pour vérifier
print(f'q_train shape: {q_train.shape}')
print(f'a_train shape: {a_train.shape}')
print(f'y_train shape: {y_train.shape}')

q_train shape: (3024, 1, 300)
a_train shape: (3024, 1, 300)
y_train shape: (3024,)


In [555]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

def create_bi_cnn_model(question_shape, answer_shape, num_features=None):
    # Define question input
    question_input = layers.Input(shape=question_shape)
    answer_input = layers.Input(shape=answer_shape)
    
    # First convolutional layer with kernel size of 1
    conv1 = layers.Conv1D(100, kernel_size=1, activation='tanh')
    
    # Second convolutional layer with kernel size of 3
    conv2 = layers.Conv1D(100, kernel_size=3, activation='tanh', padding='same')
    
    # Third convolutional layer with kernel size of 5
    conv3 = layers.Conv1D(100, kernel_size=5, activation='tanh', padding='same')
    
    # Process question and answer vectors through multiple convolutional layers
    q_conv1 = conv1(question_input)
    q_conv2 = conv2(q_conv1)
    q_conv3 = conv3(q_conv2)
    
    a_conv1 = conv1(answer_input)
    a_conv2 = conv2(a_conv1)
    a_conv3 = conv3(a_conv2)
    
    # Max pooling
    q_pool = layers.GlobalMaxPooling1D()(q_conv3)
    a_pool = layers.GlobalMaxPooling1D()(a_conv3)
    
    # Concatenate pooled outputs
    combined = layers.Concatenate()([q_pool, a_pool])
    
    if num_features is not None:
        # Add the additional features if available
        external_features = layers.Input(shape=(num_features,))
        combined_with_features = layers.Concatenate()([combined, external_features])
        
        # Fully connected layer
        dense = layers.Dense(256, activation='relu')(combined_with_features)
        dropout = layers.Dropout(0.5)(dense)
        output = layers.Dense(1, activation='sigmoid')(dropout)
        
        # Create and compile the model
        model = models.Model(inputs=[question_input, answer_input, external_features], outputs=output)
    else:
        # Fully connected layer
        dense = layers.Dense(256, activation='relu')(combined)
        dropout = layers.Dropout(0.5)(dense)
        output = layers.Dense(1, activation='sigmoid')(dropout)
        
        # Create and compile the model
        model = models.Model(inputs=[question_input, answer_input], outputs=output)
    
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Assuming q_train and a_train are your training data
question_shape = (q_train.shape[1], q_train.shape[2])  # (sequence_length, embedding_dimension)
answer_shape = (a_train.shape[1], a_train.shape[2])    # (sequence_length, embedding_dimension)

print(f'question_shape: {question_shape}')
print(f'answer_shape: {answer_shape}')

model = create_bi_cnn_model(question_shape, answer_shape, num_features=10)
model.summary()


question_shape: (1, 300)
answer_shape: (1, 300)


Model: "functional_173"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_211     │ (None, 1, 300)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_212     │ (None, 1, 300)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_109 (Conv1D) │ (None, 1, 100)    │     30,100 │ input_layer_211[… │
│                     │                   │            │ input_layer_212[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_110 (Conv1D) │ (None, 1, 100)    │     30,100 │ conv1d_109[0][0], │
│                     │                   │            │ conv1d_109[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_111 (Conv1D) │ (None, 1, 100)    │     50,100 │ conv1d_110[0][0], │
│                     │                   │            │ conv1d_110[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ conv1d_111[0][0]  │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ conv1d_111[1][0]  │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_107     │ (None, 200)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_213     │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_108     │ (None, 210)       │          0 │ concatenate_107[… │
│ (Concatenate)       │                   │            │ input_layer_213[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_173 (Dense)   │ (None, 256)       │     54,016 │ concatenate_108[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_87          │ (None, 256)       │          0 │ dense_173[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_174 (Dense)   │ (None, 1)         │        257 │ dropout_87[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 164,573 (642.86 KB)

 Trainable params: 164,573 (642.86 KB)

 Non-trainable params: 0 (0.00 B)

In [556]:
from sklearn.utils import class_weight

# Calculer les pondérations des classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights_dict = dict(enumerate(class_weights))

# Créer le modèle
model = create_bi_cnn_model(question_shape, answer_shape)

# Entraîner le modèle
history = model.fit(
    [q_train, a_train],
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    class_weight=class_weights_dict
)

Epoch 1/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.4175 - loss: 0.7004 - val_accuracy: 0.6568 - val_loss: 0.6582
Epoch 2/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5993 - loss: 0.6683 - val_accuracy: 0.6667 - val_loss: 0.5861
Epoch 3/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5916 - loss: 0.6607 - val_accuracy: 0.5776 - val_loss: 0.6405
Epoch 4/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6010 - loss: 0.6285 - val_accuracy: 0.6337 - val_loss: 0.5953
Epoch 5/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6208 - loss: 0.6241 - val_accuracy: 0.6568 - val_loss: 0.5961
Epoch 6/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6053 - loss: 0.6124 - val_accuracy: 0.5644 - val_loss: 0.6624
Epoch 7/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6482 - loss: 0.5783 - val_accuracy: 0.6007 - val_loss: 0.6735
Epoch 8/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6745 - loss: 0.5563 - val_accuracy: 0.6205 - val_los

In [557]:
q_test, a_test, y_test = prepare_data(test_grouped_clean)

# Évaluer le modèle
test_loss, test_acc = model.evaluate([q_test, a_test], y_test)
print(f'Test Accuracy: {test_acc}')

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5944 - loss: 2.3541
Test Accuracy: 0.6207650303840637


In [558]:
def predict_most_relevant_answer(model, question_vec, answer_vecs, answers):
    question_vec = np.expand_dims(np.array(question_vec), axis=0)  # (1, embedding_dimension)
    question_vec = np.expand_dims(question_vec, axis=1)  # (1, 1, embedding_dimension)
    
    answer_vecs = np.expand_dims(np.array(answer_vecs), axis=1)  # (number_of_answers, 1, embedding_dimension)
    
    # Répéter la question pour chaque réponse
    q_data = np.repeat(question_vec, len(answer_vecs), axis=0)  # (number_of_answers, 1, embedding_dimension)
    
    # Prédire les scores pour chaque réponse
    predictions = model.predict([q_data, answer_vecs])
    
    # Trouver l'indice de la réponse avec la probabilité la plus élevée
    most_relevant_index = np.argmax(predictions)
    
    # Retourner la réponse en string la plus pertinente
    most_relevant_answer = answers[most_relevant_index]
    
    return most_relevant_answer, predictions

# Exemple d'utilisation
single_question_vec = test_grouped_clean.iloc[0]['question_vec']
single_answer_vecs = test_grouped_clean.iloc[0]['answer_vecs']
single_answers = test_grouped_clean.iloc[0]['answer']
most_relevant_answer,predictions = predict_most_relevant_answer(model, single_question_vec, single_answer_vecs, single_answers)
print(f'most relevant {most_relevant_answer}')
print(f'most relevant {predictions}')
print(test_grouped_clean.iloc[0]['question'])
print(test_grouped_clean.iloc[0]['answer'])
print(test_grouped_clean.iloc[0]['label'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
most relevant a small electrically powered pump
most relevant [[0.4466947 ]
 [0.9988192 ]
 [0.36432326]
 [0.4486041 ]]
how a water pump works
['pumps operate by some mechanism typically reciprocating or rotary and consume energy to perform mechanical work by moving the fluid', 'a small electrically powered pump', 'a large electrically driven pump electropump for waterworks near the hengsteysee germany ', 'a pump is a device that moves fluids liquids or gases or sometimes slurries by mechanical action']
[1, 0, 0, 0]


In [559]:
q_val, a_val, y_val = prepare_data(validation_grouped_clean)

In [560]:
import numpy as np
q_val, a_val, y_val = prepare_data(validation_grouped_clean)

def mean_reciprocal_rank(y_true, y_pred):
    """
    Calculate Mean Reciprocal Rank (MRR).

    y_true: list of one-hot encoded true labels
    y_pred: list of one-hot encoded predicted labels

    Returns: Mean Reciprocal Rank (MRR)
    """
    ranks = []
    for true_labels, pred_labels in zip(y_true, y_pred):
        true_index = np.argmax(true_labels)
        pred_sorted_indices = np.argsort(pred_labels)[::-1]
        rank = np.where(pred_sorted_indices == true_index)[0][0] + 1  # Rank is 1-based
        ranks.append(1 / rank)
    return np.mean(ranks)


def mean_average_precision(y_true, y_pred):
    """
    Calculate Mean Average Precision (MAP).

    y_true: list of one-hot encoded true labels
    y_pred: list of one-hot encoded predicted labels

    Returns: Mean Average Precision (MAP)
    """
    average_precisions = []
    for true_labels, pred_labels in zip(y_true, y_pred):
        true_indices = np.where(true_labels == 1)[0]
        pred_sorted_indices = np.argsort(pred_labels)[::-1]
        precisions = []
        num_correct = 0
        for i, idx in enumerate(pred_sorted_indices):
            if idx in true_indices:
                num_correct += 1
                precision = num_correct / (i + 1)
                precisions.append(precision)
        if precisions:
            average_precisions.append(np.mean(precisions))
    return np.mean(average_precisions)


def success_at_1(y_true, y_pred):
    """
    Calculate Success@1 (S@1).

    y_true: list of one-hot encoded true labels
    y_pred: list of one-hot encoded predicted labels

    Returns: Success@1 (S@1)
    """
    successes = 0
    for true_labels, pred_labels in zip(y_true, y_pred):
        if np.argmax(pred_labels) == np.argmax(true_labels):
            successes += 1
    return successes / len(y_true)



# Make predictions on the validation set
y_pred = []
for q_vec, a_vecs in zip(validation_grouped_clean['question_vec'], validation_grouped_clean['answer_vecs']):
    question_vec = np.expand_dims(np.array(q_vec), axis=0)  # (1, embedding_dimension)
    question_vec = np.expand_dims(question_vec, axis=1)  # (1, 1, embedding_dimension)
    
    answer_vecs = np.expand_dims(np.array(a_vecs), axis=1)  # (number_of_answers, 1, embedding_dimension)
    
    # Répéter la question pour chaque réponse
    q_data = np.repeat(question_vec, len(answer_vecs), axis=0)  # (number_of_answers, 1, embedding_dimension)
    
    # Prédire les scores pour chaque réponse
    predictions = model.predict([q_data, answer_vecs])

    y_pred.append(predictions)
    

y_pred_onehot = []
for preds in y_pred:
    one_hot = np.zeros(len(preds))
    one_hot[np.argmax(preds)] = 1
    y_pred_onehot.append(one_hot)


# Calculate the metrics
mrr = mean_reciprocal_rank(y_val, y_pred_onehot)
map_score = mean_average_precision(y_val, y_pred_onehot)
s_at_1 = success_at_1(y_val, y_pred_onehot)


print(f'MAP: {map_score:.4f}')
print(f'MRR: {mrr:.4f}')
print(f'Success@1: {s_at_1:.4f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

C:\Users\NeilB\AppData\Local\Temp\ipykernel_18104\3124251072.py:33: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  true_indices = np.where(true_labels == 1)[0]


In [561]:
from tabulate import tabulate

def display_results(map_score, mrr, s_at_1):
    """
    Affiche les résultats dans un tableau formaté.
    
    map_score: float : Score MAP
    mrr: float : Mean Reciprocal Rank
    s_at_1: float : Success@1
    
    Returns: None
    """
    headers = ["Metric", "Score"]
    table = [
        ["Mean Average Precision (MAP)", f"{map_score:.4f}"],
        ["Mean Reciprocal Rank (MRR)", f"{mrr:.4f}"],
        ["Success@1 (S@1)", f"{s_at_1:.4f}"]
    ]
    
    print(tabulate(table, headers, tablefmt="grid"))

# Calcul des métriques
mrr = mean_reciprocal_rank(y_val, y_pred_onehot)
map_score = mean_average_precision(y_val, y_pred_onehot)
s_at_1 = success_at_1(y_val, y_pred_onehot)

# Affichage des résultats
display_results(map_score, mrr, s_at_1)


+------------------------------+---------+
| Metric                       |   Score |
+==============================+=========+
| Mean Average Precision (MAP) |  0.5793 |
+------------------------------+---------+
| Mean Reciprocal Rank (MRR)   |  0.584  |
+------------------------------+---------+
| Success@1 (S@1)              |  0.1196 |
+------------------------------+---------+


C:\Users\NeilB\AppData\Local\Temp\ipykernel_18104\3124251072.py:33: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  true_indices = np.where(true_labels == 1)[0]


## WITH FEATURES 

In [562]:
def calculate_matching_features(df):
    features = {
        'length_answer': [[length(a) for a in answers] for answers in df['answer']],
        'check_exact_match': [[check_exact_match(q, a) for a in answers] for q, answers in zip(df['question'], df['answer'])],
        'overlap': [[overlap(q, a) for a in answers] for q, answers in zip(df['question'], df['answer'])],
        'overlap_syn_fraction': [[overlap_syn_fraction(q, a) for a in answers] for q, answers in zip(df['question'], df['answer'])],
        'tagme_overlap': [[tagme_overlap(q, a) for a in answers] for q, answers in zip(df['question'], df['answer'])],
        'bm25_score': [[bm25_score(q, a, q + a) for a in answers] for q, answers in zip(df['question'], df['answer'])],
        'word2vec_similarity': [[word2vec_similarity(q, a, word2vec_model) for a in answers] for q, answers in zip(df['question'], df['answer'])]
    }
    return pd.DataFrame(features)

    # Calculer les features de lisibilité
def calculate_readability_features(df):
    features = {
        'cpw_question': [[cpw(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'spw_question': [[spw(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'wps_question': [[wps(q)  for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'cwps_question': [[cwps(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'cwr_question': [[cwr(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'lwps_question': [[lwps(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'lwr_question': [[lwr(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'dale_chall_question': [[dale_chall(q) for x in df['answer'][i] ] for i,q in enumerate(df['question'])],
        'cpw_answer': [[cpw(a) for a in answers] for answers in df['answer']],
        'spw_answer': [[spw(a) for a in answers] for answers in df['answer']],
        'wps_answer': [[wps(a) for a in answers] for answers in df['answer']],
        'cwps_answer': [[cwps(a) for a in answers] for answers in df['answer']],
        'cwr_answer': [[cwr(a) for a in answers] for answers in df['answer']],
        'lwps_answer': [[lwps(a) for a in answers] for answers in df['answer']],
        'lwr_answer': [[lwr(a) for a in answers] for answers in df['answer']],
        'dale_chall_answer': [[dale_chall(a) for a in answers] for answers in df['answer']]
    }
    return pd.DataFrame(features)

# Fusionner les fonctionnalités de lisibilité et de matching lexical
def calculate_all_features(df):
    readability_features = calculate_readability_features(df)
    matching_features = calculate_matching_features(df)
    return pd.concat([readability_features, matching_features], axis=1)

# Calculer les fonctionnalités pour les ensembles de données groupés

train_grouped_all_features = pd.concat([train_grouped_clean, calculate_all_features(train_grouped_clean)], axis=1)
validation_grouped_all_features = pd.concat([validation_grouped_clean, calculate_all_features(validation_grouped_clean)], axis=1)
test_grouped_all_features = pd.concat([test_grouped_clean, calculate_all_features(test_grouped_clean)], axis=1)

In [563]:
features_list = [
    'cpw_question', 'spw_question', 'wps_question', 'cwps_question',
    'cwr_question', 'lwps_question', 'lwr_question', 'dale_chall_question',
    'cpw_answer', 'spw_answer', 'wps_answer', 'cwps_answer', 'cwr_answer',
    'lwps_answer', 'lwr_answer', 'dale_chall_answer', 'length_answer',
    'check_exact_match', 'overlap', 'overlap_syn_fraction', 'tagme_overlap',
    'bm25_score', 'word2vec_similarity'
]

# Extract the features
features = train_grouped_all_features[features_list]
features

,cpw_question,spw_question,wps_question,cwps_question,cwr_question,lwps_question,lwr_question,dale_chall_question,cpw_answer,spw_answer,...,lwps_answer,lwr_answer,dale_chall_answer,length_answer,check_exact_match,overlap,overlap_syn_fraction,tagme_overlap,bm25_score,word2vec_similarity
0,"[4.2, 4.2, 4.2, 4.2]","[1.4, 1.4, 1.4, 1.4]","[5, 5, 5, 5]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]","[7.04, 7.04, 7.04, 7.04]","[4.608695652173913, 4.75, 5.636363636363637, 4...","[1.4782608695652173, 1.5, 1.8181818181818181, ...",...,"[2, 0, 4, 0]","[0.08695652173913043, 0.0, 0.18181818181818182...","[10.27, 11.93, 10.47, 11.73]","[23, 8, 22, 4]","[0, 0, 0, 0]","[0.2608695652173913, 0.0, 0.0, 0.5]","[0.2608695652173913, 0.0, 0.0, 0.5]","[0.0, 0.0, 0.0, 0.0]","[2.1695319202612433, 0.0, 0.0, 3.425402019863292]","[0.7429044, 0.2997008, 0.5121894, 0.71078146]"
1,"[5.333333333333333, 5.333333333333333, 5.33333...","[1.6666666666666667, 1.6666666666666667, 1.666...","[6, 6, 6, 6]","[1, 1, 1, 1]","[0.16666666666666666, 0.16666666666666666, 0.1...","[1, 1, 1, 1]","[0.16666666666666666, 0.16666666666666666, 0.1...","[11.83, 11.83, 11.83, 11.83]","[5.407407407407407, 5.5, 6.2, 4.4]","[1.7407407407407407, 2.0, 2.2, 1.8]",...,"[6, 1, 2, 0]","[0.2222222222222222, 0.16666666666666666, 0.4,...","[10.82, 14.46, 10.2, 10.2]","[27, 6, 5, 5]","[0, 0, 0, 0]","[0.037037037037037035, 0.16666666666666666, 0....","[0.1111111111111111, 0.3333333333333333, 0.0, ...","[0.0, 0.0, 0.0, 0.0]","[0.40768426907095634, 1.3706284736831396, 0.0,...","[0.7085388, 0.61963964, 0.2614088, 0.70052916]"
2,"[5.625, 5.625, 5.625, 5.625]","[1.625, 1.625, 1.625, 1.625]","[8, 8, 8, 8]","[2, 2, 2, 2]","[0.25, 0.25, 0.25, 0.25]","[2, 2, 2, 2]","[0.25, 0.25, 0.25, 0.25]","[11.93, 11.93, 11.93, 11.93]","[5.392857142857143, 5.214285714285714, 5.375, ...","[1.8214285714285714, 1.5714285714285714, 1.75,...",...,"[8, 4, 3, 6]","[0.2857142857142857, 0.2857142857142857, 0.375...","[11.79, 12.23, 13.9, 10.94]","[28, 14, 8, 20]","[0, 0, 0, 0]","[0.0, 0.07142857142857142, 0.375, 0.3]","[0.0, 0.07142857142857142, 0.375, 0.3]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.7068947195130464, 3.3614266532470127, ...","[0.5874782, 0.66302633, 0.51851976, 0.53118145]"
3,"[4.8, 4.8, 4.8, 4.8]","[1.0, 1.0, 1.0, 1.0]","[5, 5, 5, 5]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]","[13.36, 13.36, 13.36, 13.36]","[3.6153846153846154, 5.333333333333333, 4.5714...","[1.0769230769230769, 1.4444444444444444, 1.714...",...,"[0, 1, 1, 1]","[0.0, 0.1111111111111111, 0.14285714285714285,...","[7.93, 11.1, 10.75, 10.75]","[13, 9, 7, 7]","[0, 0, 0, 0]","[0.23076923076923078, 0.2222222222222222, 0.0,...","[0.38461538461538464, 0.3333333333333333, 0.0,...","[0.0, 0.0, 0.0, 0.0]","[1.9280123653507184, 1.7962903299676172, 0.0, ...","[0.8436012, 0.7371161, 0.39387175, 0.7742784]"
4,"[4.555555555555555, 4.555555555555555, 4.55555...","[1.3333333333333333, 1.3333333333333333, 1.333...","[9, 9, 9, 9]","[1, 1, 1, 1]","[0.1111111111111111, 0.1111111111111111, 0.111...","[2, 2, 2, 2]","[0.2222222222222222, 0.2222222222222222, 0.222...","[7.59, 7.59, 7.59, 7.59]","[3.6222222222222222, 6.2, 4.277777777777778, 3...","[1.1333333333333333, 2.0, 1.3333333333333333, ...",...,"[3, 2, 3, 1]","[0.06666666666666667, 0.4, 0.16666666666666666...","[8.32, 13.36, 10.67, 8.49]","[45, 5, 18, 36]","[0, 0, 0, 0]","[0.28888888888888886, 0.0, 0.16666666666666666...","[0.28888888888888886, 0.2, 0.2777777777777778,...","[0.0, 0.0, 0.0, 0.0]","[3.0127938438548734, 0.0, 1.630096521347441, 3...","[0.82951623, 0.6748694, 0.7648825, 0.67787033]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,"[4.1, 4.1, 4.1, 4.1]","[1.2, 1.2, 1.2, 1.2]","[10, 10, 10, 10]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0]","[0.5, 0.5, 0.5, 0.5]","[4.333333333333333, 4.647058823529412, 4.26666...","[1.393939393939394, 1.588235294117647, 1.3, 1....",...,"[3, 3, 

### Prepare DATA

In [564]:
def extract_features(row):
    feature_row = []
    for col in [
        'cpw_question', 'spw_question', 'wps_question', 'cwps_question',
        'cwr_question', 'lwps_question', 'lwr_question', 'dale_chall_question',
        'cpw_answer', 'spw_answer', 'wps_answer', 'cwps_answer', 'cwr_answer',
        'lwps_answer', 'lwr_answer', 'dale_chall_answer', 'length_answer',
        'check_exact_match', 'overlap', 'overlap_syn_fraction', 'tagme_overlap',
        'bm25_score', 'word2vec_similarity'
    ]:
        feature_row.append(row[col])
    return np.array(feature_row)

train_grouped_clean['features'] = train_grouped_all_features.apply(extract_features, axis=1)
validation_grouped_clean['features'] = validation_grouped_all_features.apply(extract_features, axis=1)
test_grouped_clean['features'] = test_grouped_all_features.apply(extract_features, axis=1)



def prepare_data(df):
    q_data = []
    a_data = []
    y_data = []
    features_data = []

    for _, row in df.iterrows():
        question_vec = np.array(row['question_vec'])
        answer_vecs = np.array(row['answer_vecs'])
        labels = row['label']
        features = np.array(row['features']).reshape(len(labels), -1)  # Ensure features are properly shaped

        num_answers = len(answer_vecs)
        q_data.extend([question_vec] * num_answers)
        a_data.extend(answer_vecs)
        y_data.extend(labels)
        features_data.extend(features)

    q_data = np.expand_dims(np.array(q_data), axis=1)  # Add an axis for the sequence
    a_data = np.expand_dims(np.array(a_data), axis=1)  # Add an axis for the sequence
    y_data = np.array(y_data)
    features_data = np.array(features_data)

    return q_data, a_data, y_data, features_data

q_train, a_train, y_train, features_train = prepare_data(train_grouped_clean)
q_val, a_val, y_val, features_val = prepare_data(validation_grouped_clean)
q_test, a_test, y_test, features_test = prepare_data(test_grouped_clean)

# Print shapes to verify
print(f'q_train shape: {q_train.shape}')
print(f'a_train shape: {a_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'features_train shape: {features_train.shape}')


q_train shape: (3024, 1, 300)
a_train shape: (3024, 1, 300)
y_train shape: (3024,)
features_train shape: (3024, 23)


In [565]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

def create_bi_cnn_model(question_shape, answer_shape, num_features=None):
    # Define question input
    question_input = layers.Input(shape=question_shape)
    answer_input = layers.Input(shape=answer_shape)
    
    # First convolutional layer with kernel size of 1
    conv1 = layers.Conv1D(100, kernel_size=1, activation='tanh')
    
    # Second convolutional layer with kernel size of 3
    conv2 = layers.Conv1D(100, kernel_size=3, activation='tanh', padding='same')
    
    # Third convolutional layer with kernel size of 5
    conv3 = layers.Conv1D(100, kernel_size=5, activation='tanh', padding='same')
    
    # Process question and answer vectors through multiple convolutional layers
    q_conv1 = conv1(question_input)
    q_conv2 = conv2(q_conv1)
    q_conv3 = conv3(q_conv2)
    
    a_conv1 = conv1(answer_input)
    a_conv2 = conv2(a_conv1)
    a_conv3 = conv3(a_conv2)
    
    # Max pooling
    q_pool = layers.GlobalMaxPooling1D()(q_conv3)
    a_pool = layers.GlobalMaxPooling1D()(a_conv3)
    
    # Concatenate pooled outputs
    combined = layers.Concatenate()([q_pool, a_pool])
    
    if num_features is not None:
        # Add the additional features if available
        external_features = layers.Input(shape=(num_features,))
        combined_with_features = layers.Concatenate()([combined, external_features])
        
        # Fully connected layer
        dense = layers.Dense(256, activation='relu')(combined_with_features)
        dropout = layers.Dropout(0.5)(dense)
        output = layers.Dense(1, activation='sigmoid')(dropout)
        
        # Create and compile the model
        model = models.Model(inputs=[question_input, answer_input, external_features], outputs=output)
    else:
        # Fully connected layer
        dense = layers.Dense(256, activation='relu')(combined)
        dropout = layers.Dropout(0.5)(dense)
        output = layers.Dense(1, activation='sigmoid')(dropout)
        
        # Create and compile the model
        model = models.Model(inputs=[question_input, answer_input], outputs=output)
    
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Determine the shapes of the inputs
question_shape = (q_train.shape[1], q_train.shape[2])  # (sequence_length, embedding_dimension)
answer_shape = (a_train.shape[1], a_train.shape[2])    # (sequence_length, embedding_dimension)
num_features = features_train.shape[1]

print(f'question_shape: {question_shape}')
print(f'answer_shape: {answer_shape}')
print(f'num_features: {num_features}')

model = create_bi_cnn_model(question_shape, answer_shape, num_features)
model.summary()


question_shape: (1, 300)
answer_shape: (1, 300)
num_features: 23


Model: "functional_177"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_216     │ (None, 1, 300)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_217     │ (None, 1, 300)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_115 (Conv1D) │ (None, 1, 100)    │     30,100 │ input_layer_216[… │
│                     │                   │            │ input_layer_217[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_116 (Conv1D) │ (None, 1, 100)    │     30,100 │ conv1d_115[0][0], │
│                     │                   │            │ conv1d_115[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_117 (Conv1D) │ (None, 1, 100)    │     50,100 │ conv1d_116[0][0], │
│                     │                   │            │ conv1d_116[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ conv1d_117[0][0]  │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ conv1d_117[1][0]  │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_110     │ (None, 200)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_218     │ (None, 23)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_111     │ (None, 223)       │          0 │ concatenate_110[… │
│ (Concatenate)       │                   │            │ input_layer_218[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_177 (Dense)   │ (None, 256)       │     57,344 │ concatenate_111[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_89          │ (None, 256)       │          0 │ dense_177[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_178 (Dense)   │ (None, 1)         │        257 │ dropout_89[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 167,901 (655.86 KB)

 Trainable params: 167,901 (655.86 KB)

 Non-trainable params: 0 (0.00 B)

In [566]:
from sklearn.utils import class_weight

# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights_dict = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    [q_train, a_train, features_train],
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=([q_val, a_val, features_val], y_val),
    class_weight=class_weights_dict
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([q_test, a_test, features_test], y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8263 - loss: 0.8011 - val_accuracy: 0.8630 - val_loss: 0.8652
Epoch 2/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9242 - loss: 0.2874 - val_accuracy: 0.8565 - val_loss: 0.9529
Epoch 3/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9067 - loss: 0.2612 - val_accuracy: 0.8630 - val_loss: 1.1130
Epoch 4/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9405 - loss: 0.1639 - val_accuracy: 0.8652 - val_loss: 1.1175
Epoch 5/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9520 - loss: 0.1452 - val_accuracy: 0.8630 - val_loss: 1.1919
Epoch 6/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9399 - loss: 0.1583 - val_accuracy: 0.8609 - val_loss: 1.2887
Epoch 7/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9591 - loss: 0.1149 - val_accuracy: 0.8739 - val_loss: 1.4369
Epoch 8/50
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9568 - loss: 0.1166 - val_accuracy: 0.8696 - val_loss:

In [567]:
import numpy as np
import torch

q_val, a_val, y_val, features_val = prepare_data(validation_grouped_clean)


def mean_reciprocal_rank(y_true, y_pred):
    """Compute the Mean Reciprocal Rank (MRR)"""
    ranks = []
    for true, pred in zip(y_true, y_pred):
        sorted_pred = np.argsort(pred)[::-1]
        print(y_true)
        rank = np.where(sorted_pred == true)[0][0] + 1
        ranks.append(1 / rank)
    return np.mean(ranks)

def mean_average_precision(y_true, y_pred):
    """Compute the Mean Average Precision (MAP)"""
    average_precisions = []
    for true, pred in zip(y_true, y_pred):
        sorted_pred = np.argsort(pred)[::-1]
        hits = 0
        sum_precisions = 0
        for i, p in enumerate(sorted_pred):
            if p == true:
                hits += 1
                sum_precisions += hits / (i + 1)
        average_precisions.append(sum_precisions / hits if hits > 0 else 0)
    return np.mean(average_precisions)

def success_at_1(y_true, y_pred):
    """Compute the Success@1 (S@1)"""
    successes = 0
    for true, pred in zip(y_true, y_pred):
        if np.argmax(pred) == true:
            successes += 1
    return successes / len(y_true)

# Make predictions on the validation set
y_pred = []
for q_vec, a_vecs, features in zip(validation_grouped_clean['question_vec'], validation_grouped_clean['answer_vecs'], validation_grouped_clean['features']):
    question_vec = np.expand_dims(np.array(q_vec), axis=0)  # (1, embedding_dimension)
    question_vec = np.expand_dims(question_vec, axis=1)  # (1, 1, embedding_dimension)
    
    answer_vecs = np.expand_dims(np.array(a_vecs), axis=1)  # (number_of_answers, 1, embedding_dimension)
    
    # Repeat the question for each answer
    q_data = np.repeat(question_vec, len(answer_vecs), axis=0)  # (number_of_answers, 1, embedding_dimension)
    
    # Extract the corresponding features for each answer
    features_data = np.expand_dims(features, axis=1)
    features_data = np.array(features).reshape(len(answer_vecs), -1)
    
    print(f"answer_vecs shape = {answer_vecs.shape}")
    print(f"q_data shape = {q_data.shape}")
    print(f"features_data shape = {features_data.shape}")
    
    # Predict scores for each answer
    predictions = model.predict([q_data, answer_vecs, features_data])
    y_pred.append(predictions)

# Calculate the metrics
mrr = mean_reciprocal_rank(validation_grouped_clean['label'], y_pred)
map_score = mean_average_precision(validation_grouped_clean['label'], y_pred)
s_at_1 = success_at_1(validation_grouped_clean['label'], y_pred)


print(f'MAP: {map_score:.4f}')
print(f'MRR: {mrr:.4f}')
print(f'Success@1: {s_at_1:.4f}')


answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━

answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
answer_vecs shape = (3, 1, 300)
q_data shape = (3, 1, 300)
features_data shape = (3, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
answer_vecs shape = (3, 1, 300)
q_data shape = (3, 1, 300)
features_data shape = (3, 23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
answer_vecs shape = (4, 1, 300)
q_data shape = (4, 1, 300)
features_data shape = (4, 23)
1/1 ━━━━━━━━━━━━━━━━━━

IndexError: index 0 is out of bounds for axis 0 with size 0